<a href="https://colab.research.google.com/github/deea-c/thesis_absa/blob/main/Hyperparameter_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/code
!pwd 

/content/drive/MyDrive/Colab Notebooks/code
/content/drive/MyDrive/Colab Notebooks/code


In [ ]:
#To install
!pip install evaluate
!pip install seqeval (need to install this) 
!pip install optuna
!pip install transformers
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 23.4 MB/s eta 0:00:00
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `pip ins

In [ ]:
from transformers import set_seed
random_seed = 11
set_seed(random_seed)

In [ ]:
#Packages
from help_functions import get_annotations_as_dict, align_tokens_and_annotations_bio, compute_metrics
from help_functions import metric
from help_functions import TraingDataset, LabelSet, TrainingExample, label_list, label_set
import pandas as pd 
import numpy as np
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer,EarlyStoppingCallback
from transformers import DataCollatorForTokenClassification
from sklearn.model_selection import train_test_split
import evaluate

In [ ]:
#Load data
df = get_annotations_as_dict( pd.read_pickle("../pickles/" + "train_English" +".pkl"))
test_df = get_annotations_as_dict( pd.read_pickle("../pickles/" + "test_English" +".pkl"))

In [ ]:
model_name = 'bert-base-multilingual-cased'


tokenizer = AutoTokenizer.from_pretrained(model_name) # Load a pre-trained tokenizer
label_set = LabelSet(labels=["positive", "negative", "neutral"])

ds = TraingDataset(
    data=df, tokenizer=tokenizer, label_set=label_set, tokens_per_batch=89)
test_ds = TraingDataset(
    data=test_df, tokenizer=tokenizer, label_set=label_set, tokens_per_batch=89)

train_ds , val_ds= train_test_split(ds, test_size=0.2, random_state=random_seed)
data_collator = DataCollatorForTokenClassification(tokenizer)
label_list = label_set.ids_to_label
print(label_list)

{0: 'O', 1: 'B-positive', 2: 'I-positive', 3: 'B-negative', 4: 'I-negative', 5: 'B-neutral', 6: 'I-neutral'}


In [ ]:
import json

In [ ]:

training_args = TrainingArguments(
    output_dir = f"search/{model_name}-finetuned",
    logging_dir= f"search/{model_name}-finetuned/log",
    overwrite_output_dir = True,
    evaluation_strategy = "epoch",
    logging_strategy= "epoch",
    save_strategy = "epoch",
    per_device_eval_batch_size=16,   
    num_train_epochs =10, 
    metric_for_best_model  = "eval_loss",
    greater_is_better = False,
    seed = 11,
    save_total_limit=1,
    push_to_hub= False,
    load_best_model_at_end = True
)

def model_init():
    model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_set.labels_to_id), label2id = label_set.labels_to_id, id2label =label_set.ids_to_label )
    for param in model.base_model.parameters():
        param.requires_grad = True
    return model
trainer = Trainer(
    args=training_args,   
    train_dataset= train_ds,
    eval_dataset= val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    model_init=model_init,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
   )

def my_hp_space(trial):
    learning_rates = [5e-5, 3e-5, 2e-5]
    w_d =  [1e-1,1e-2,1e-3,1e-5 ]
    batch_t =[8, 16, 32]
    return {
        'learning_rate': trial.suggest_categorical('learning_rate', learning_rates),
        'per_device_train_batch_size': trial.suggest_categorical('per_device_train_batch_size', batch_t),
        'weight_decay': trial.suggest_categorical('weight_decay',w_d),
    }
def my_objective(metrics):
    return metrics["eval_val_f1"]

best_run = trainer.hyperparameter_search(direction="maximize",backend="optuna", hp_space=my_hp_space, compute_objective=my_objective, n_trials=30)




Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

Epoch,Training Loss,Validation Loss,Val Precision,Val Recall,Val F1,Val Accuracy
1,0.495600,0.346922,0.328283,0.168394,0.222603,0.881513
2,0.273800,0.265320,0.475728,0.507772,0.491228,0.913488
3,0.201700,0.262916,0.461347,0.479275,0.470140,0.913975
4,0.161800,0.258950,0.502155,0.603627,0.548235,0.917708
5,0.129100,0.268842,0.514019,0.569948,0.540541,0.920467
6,0.106700,0.283332,0.514286,0.606218,0.556480,0.921604


[I 2023-05-09 16:35:39,685] Trial 0 finished with value: 0.5564803804994055 and parameters: {'learning_rate': 2e-05, 'per_device_train_batch_size': 32, 'weight_decay': 0.001}. Best is trial 0 with value: 0.5564803804994055.
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that y

Epoch,Training Loss,Validation Loss,Val Precision,Val Recall,Val F1,Val Accuracy
1,0.495600,0.346921,0.328283,0.168394,0.222603,0.881513
2,0.273800,0.265315,0.475728,0.507772,0.491228,0.913488
3,0.201700,0.262917,0.461347,0.479275,0.470140,0.913975
4,0.161800,0.258947,0.502155,0.603627,0.548235,0.917708
5,0.129100,0.268843,0.514019,0.569948,0.540541,0.920467
6,0.106700,0.283331,0.514286,0.606218,0.556480,0.921604


[I 2023-05-09 16:39:29,483] Trial 1 finished with value: 0.5564803804994055 and parameters: {'learning_rate': 2e-05, 'per_device_train_batch_size': 32, 'weight_decay': 0.01}. Best is trial 0 with value: 0.5564803804994055.


In [ ]:
with open(f"./search/modelbest_run.json", "w+") as f:
  f.write(json.dumps(best_run.hyperparameters))